In [1]:
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter




# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
# 문서 로드
loader = TextLoader("./data/text.txt", encoding="utf-8")
# 문서 분할
split_docs = loader.load_and_split(text_splitter)

# 임베딩 모델 설정
model_name = "intfloat/multilingual-e5-large-instruct"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

# 문서로부터 FAISS
db = FAISS.from_documents(documents=split_docs, embedding=hf_embeddings)

    # 5. 리트리버 생성
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25, "fetch_k": 10}
)

# 질의 설정
query = "삼성전자 하반기 실적 전망은?"

# 질의 수행 및 결과 문서 반환
docs = retriever.invoke(query)

# 결과 문서 출력
pretty_print_docs(docs)

Document 1:

특히 파운드리 고객사를 확보한 것이 단기 실적에는 영향을 미치지 않지만, 중장기 경쟁력 회복을 위한 기반이 될 것이라고 평가했다. 류 연구원은 "여러 기대감이 모여 밸류에이션이 정상화되는 국면"이라고 분석했다. 
이어 3분기 영업이익은 9조원으로 1년 전보다는 1.9% 감소하지만 직전 분기 대비 91.9% 늘어날 것으로 낙관했다.
미래에셋증권은 15일 보고서에서 삼성전자의 올해와 내년 영업이익 전망치를 기존대비 각각 2.9%, 5.3%씩 올렸다. 동시에 목표 주가를 기존 8만8000원에서 9만6000원으로 9% 높였다.
미래에셋증권은 주가 상향 이유로 내년 메모리 반도체 공급 부족을 꼽았다. 반도체 업체들은 제한된 D램 생산라인을 갖고 있는데, 업황 불확실성 탓에 설비를 크게 늘리지 못했다. 새로 증설한 시설도 대부분 차세대 반도체인 고대역폭 메모리(HBM) 생산에 집중돼 있어 범용 D램의 생산량이 줄면서 가격이 오르고 있다.
----------------------------------------------------------------------------------------------------
Document 2:

경기도 이천시 SK하이닉스 본사 모습. 이천=연합뉴스
올해 상반기 16조6,000억 원대로 국내 1위 영업이익을 달성한 SK하이닉스가 같은 기간 국내 법인세도 가장 많이 낸 것으로 나타났다.
----------------------------------------------------------------------------------------------------
Document 3:

Q. 2010년대 노키아 몰락후 핀란드가 크게 흔들렸습니다. TSMC 의존도가 절대적으로 높은 대만도 그런 위험에 빠질 가능성이 있을까요?
Q. 2027년 중국의 대만 침공설이 나돌 정도로 중국은 대만 경제 의 앞날에 최대 리스크입니다. 대만내에서는 중국 침공 가능성을 어떻게 보고 있나요?
Q. 한국은 10년째 3만달러 박스에

: 